<br/>
<br/>
<br/>
<br/>
<center><b><span style="font-size:2.7em"> MIT IAP Challenge - C$ Demand Simulation <br/> <br/>
    <br/> <br/> </span></b></center> 

<img src="https://3mgj4y44nc15fnv8d303d8zb-wpengine.netdna-ssl.com/wp-content/uploads/2018/06/celo-696x449.jpg" align="center" style="width:50%"/>
<center><a href="roman@celo.org">roman@celo.org</a>
<br/>
<br/>
<br/>
<br/>
<br/>


In [1]:
# import modules and set up the notebook
import numpy as np
import pandas as pd
import plotly
from IPython.display import display, HTML
from plots import plotly_from_df
from stat_utils import gbm
# makes offline plotly plots appear in notebook
plotly.offline.init_notebook_mode(connected=True)

# Generate Celo Dollar Demand From Piecewise GBM

In [11]:
# GBM parameters
initial_student_total_supply = 20 * 50  # assuming 50 students and 25 C$ each (+ 25 CG per student)
cd_demand_start = 10 * initial_student_total_supply  # Students hold 10% initially, demand_0 = supply_0
mu = 2  # Does not really matter as we pick out paths with a specific realized drift rate anyway

# Sigmas for 5 day periods, over 30 days total (longer than needed)
sigma = []
sigma.append(0.4)  
sigma.append(0.8)  
sigma.append(1.2)  
sigma.append(0.8)  
sigma.append(0.8) 
sigma.append(0.8) 

dt = 1/365/24/60  # step size is one minute
steps_per_day = int(1 / dt / 365)  #
N = 2500  # how many paths to select from
seed = 6  # fix seed to allow reproducability

print('Total initial C$ supply: ' + str(cd_demand_start))
print('Initial C$ supply of students: ' + str(initial_student_total_supply))
print('Initial total (C$ + CG) portfolio value of students: ' + str(initial_student_total_supply*2))
print('Initial C$ supply in Uniswap: ' + str(initial_student_total_supply * 8))
print('Initial C$ supply of Whale (not supposed to act currently): ' + str(initial_student_total_supply))

Total initial C$ supply: 10000
Initial C$ supply of students: 1000
Initial total (C$ + CG) portfolio value of students: 2000
Initial C$ supply in Uniswap: 8000
Initial C$ supply of Whale (not supposed to act currently): 1000


In [3]:
# Simulate piecewise GBM demand series
this_cd_demand = np.array([])
x0 = [cd_demand_start]*N
for current_sigma in sigma:
    if seed != 0:
        seed += 1
    seed +=1
    cd_demand = gbm(x0=x0, mu=[mu]*N, cov=current_sigma * np.eye(N), dt=dt, steps=steps_per_day * 5, sim_iter=1, randomize=seed)
    x0 = cd_demand[0][-1,:] 
    if this_cd_demand.shape[0] == 0:
        this_cd_demand = cd_demand[0]
    else:
        this_cd_demand = np.concatenate((this_cd_demand, cd_demand[0]), axis=0)
    
cd_demand_df = pd.DataFrame(this_cd_demand)

# Add black swan event (about 1/3 of cd demand is lost)
cd_returns = cd_demand_df.pct_change()
cd_returns.loc[int(steps_per_day * 15.5), :] = -0.2230 
gross_returns = cd_returns + 1
gross_returns.iloc[0,:] = [cd_demand_start] * N

cd_demand_df = gross_returns.cumprod()


In [4]:
# Set properties of paths of interest
cd_demand_df = cd_demand_df.loc[:, cd_demand_df.loc[steps_per_day*20, :] < 1.7 * cd_demand_start]
cd_demand_df = cd_demand_df.loc[:, cd_demand_df.loc[steps_per_day*20, :] > 1.4 * cd_demand_start]
cd_demand_df = cd_demand_df.loc[:, cd_demand_df.max() > 1.8 * cd_demand_start]
cd_demand_df = cd_demand_df.loc[:, cd_demand_df.max() < 2.2 * cd_demand_start]
cd_demand_df = cd_demand_df.loc[:, cd_demand_df.min() < 0.95 * cd_demand_start]

cd_demand_df.index = cd_demand_df.index.values / steps_per_day # translate index to days

In [5]:
# Get expansion/contraction stats
cd_demand_df_diff = cd_demand_df.iloc[0::int(steps_per_day/24),:].diff()
cd_demand_df_diff.describe().T

,count,mean,std,min,25%,50%,75%,max
1855,720.0,12.233004,173.880953,-3102.676921,-51.682120,14.855660,85.714925,383.782295
2264,720.0,10.977742,213.173102,-4159.964853,-79.265802,17.838921,103.327674,462.547874


In [6]:
# Show both exchange rates after arbitrage trades
iplot_dict = plotly_from_df(cd_demand_df.iloc[0::60,:], title='C$ Demand', xlabel='Day', mode='lines')
plotly.offline.iplot(iplot_dict)

In [7]:
# Select path 
iap_challenge_path = 2264


In [8]:
# Histogram of expansion amounts (negative amounts are contractions) - assumes we are always able to expand/contract full amount
data = [plotly.graph_objs.Histogram(x=cd_demand_df_diff.loc[:,iap_challenge_path])]
plotly.offline.iplot(data, filename='Histogram of Expansion Amounts')

In [9]:
# Cum relative frequency of Expansion amounts (negative amounts are contractions) - assumes we are always able to expand/contract full amount
data = [plotly.graph_objs.Histogram(x=cd_demand_df_diff.loc[:,iap_challenge_path],histnorm='probability',
                     cumulative=dict(enabled=True))]

plotly.offline.iplot(data, filename='Cumulative Relative Frequency of Expansion Amounts')

In [13]:
# Final student portfolio value if they buy and sell at discount to uniswap in hourly auctions
discount = 0.05  # 0.05 means 5% discount
total_gain_from_exp_and_cont = (cd_demand_df_diff.loc[:,iap_challenge_path].abs() * discount).sum()
print('Total gain through auction discount (in C$ units): ' + str(total_gain_from_exp_and_cont))
print('(Very rough) estimate of portfolio value prize money to distribute (in USD): ' + str(initial_student_total_supply*2 + total_gain_from_exp_and_cont))

Total gain through auction discount (in C$ units): 4301.2015268360465
(Very rough) estimate of portfolio value prize money to distribute (in USD): 6301.2015268360465
